In [ ]:
#@ All plotting functions
def plot_pca_comps(X_transformed ,angles):
    """Plot all the principal components with respect to the orientation angles
    
    Args:
    X_transformed (numpy.ndarray): principle components (transformed sresp)
    angles (numpy.ndarray): stimulus orientation angles
  
    Returns:
    None
    """
    #@ Plot PC response to stimulus angle
    from matplotlib import pyplot as plt
    from scipy.signal import butter, filtfilt

    b,a = butter(4, [0.01], 'lowpass') #creating a low pass butterworth filter

    idx = np.argsort(angles)
    fig = plt.figure(figsize = (15,10))
    for j in np.arange(n_cmp):
        fig.add_subplot(4,5,j+1,polar = True)
        plt.plot(angles[idx],filtfilt(b,a,X_transformed[j,idx]))
    plt.show()
    return None

def plot_pca_modes(X_transformed ,angles, mode_inds):
    """Plot principal components with respect to the orientation angles as 
    
    Args:
    X_transformed (numpy.ndarray): principle components (transformed sresp)
    angles (numpy.ndarray): stimulus orientation angles
    mode_inds (dict): dictionary with mode keys and PC numbers
  
    Returns:
    None
    """
    from matplotlib import pyplot as plt
    from scipy.signal import butter, filtfilt
    b,a = butter(4, [0.01], 'lowpass') #creating a low pass butterworth filter
    idx = np.argsort(angles)
    fig = plt.figure(figsize = (15,6))
    for j in np.arange(len(mode_inds)):
        for k in np.arange(len(mode_inds[j+1])):
            fig.add_subplot(2, len(mode_inds),len(mode_inds)*(k) + j +1,polar = True)
            if k%2==0:
                plt.title('Mode %d\n' %(j+1))
            if isinstance(mode_inds[j+1][k],int):
                tmp = X_transformed[mode_inds[j+1][k],idx]
            else:
                tmp = np.zeros(X_transformed.shape[1])
                for m in mode_inds[j+1][k]:
                    tmp = tmp + X_transformed[m,idx]
            plt.plot(angles[idx],filtfilt(b,a,tmp))
#         plt.gca().set_rlim([0,40])
    plt.show()
    return None

def plot_neurons(Z2 , dat2 ,p2, compN, n_start = 0, n_stop = 100, n_step = 1, ydel = 2e1, yupperlim = 100):
    """Plot angle responses of neaurons of one principal component
    
    Args:
    Z2 (numpy.ndarray): Z-scored neural response
    dat2 (dict): loaded stringer dataset
    p2 (PCA model): PCA model based on Z2
    compN (int): Principal component number to sort neurons by their weights
  
    Returns:
    None
    """
    from matplotlib import pyplot as plt
    from scipy.signal import butter, filtfilt
    from scipy.stats import zscore
    b,a = butter(4, [0.1], 'lowpass')
    fig = plt.figure(figsize = (14,10))
    pidx = np.argsort((-p2.components_[compN,:]))
    alpha = 0.6
    poscol = [0,0,0.5,alpha]
    negcol = [0.5,0,0,alpha]
    poscol, negcol = [0,0,0,alpha], [0,0,0,alpha]
    idx = np.argsort(dat2['istim'])
    t = np.linspace(0,2*np.pi,len(idx))
    plt.subplot(1,3,1)
    plt.title('Highest Weights',fontsize = 16)
    Z2_sum1 = np.zeros((Z2.shape[1]))
    for k , k2 in enumerate(np.arange(n_start,n_stop,n_step)):
        if p2.components_[compN,pidx[k2]] >=0:
            col = poscol
        else:
            col = negcol
        plt.plot(t,Z2[pidx[k2],idx] + k*ydel, color = col)
        Z2_sum1 = Z2_sum1 + Z2[pidx[k2],idx]
        Z2_sum1 = filtfilt(b,a,Z2_sum1)
    plt.plot(t,(Z2_sum1)*40/((n_stop - n_start)/n_step) + (k+1.5)*ydel, color = [0.5,0,0])
#     plt.ylim([n_start*ydel,(n_stop - n_start)*ydel/n_step])
    plt.ylim([n_start*ydel,(n_stop - n_start + yupperlim)*ydel/n_step])
    plt.xlim([0,2*np.pi])
    plt.xticks(ticks = np.array([0,0.5,1,1.5,2])*np.pi, labels = ['0','$\\frac{\pi}{2}$','$\pi$', '$\\frac{\pi}{2}$', '$2\pi$'],fontsize = 14)
    plt.yticks([])
    plt.xlabel('Orientation Angle', fontsize = 14)
    plt.ylabel('Sorted Neurons', fontsize = 14)

    plt.subplot(1,3,2)
    plt.title('Middle Weights',fontsize = 16)
    Z2_sum2 = np.zeros((Z2.shape[1]))
    for k , k2 in enumerate(np.arange(n_start,n_stop,n_step)):
        if p2.components_[compN,pidx[k2]] >=0:
            col = poscol
        else:
            col = negcol
        plt.plot(t,Z2[pidx[k2 + np.round(len(pidx)/2 - (n_stop-n_start)/2).astype(int)],idx] + k*ydel, color = col)
        Z2_sum2 = Z2_sum2 + Z2[pidx[k2 + np.round(len(pidx)/2 - (n_stop-n_start)/2).astype(int)],idx]
        Z2_sum2 = filtfilt(b,a,Z2_sum2)
    plt.plot(t,(Z2_sum2)*40/((n_stop - n_start)/n_step) + (k+1.5)*ydel, color = [0.5,0,0])
#     plt.ylim([n_start*ydel,(n_stop - n_start)*ydel/n_step])
    plt.ylim([n_start*ydel,(n_stop - n_start + yupperlim)*ydel/n_step])
    plt.xlim([0,2*np.pi])
    plt.xticks(ticks = np.array([0,0.5,1,1.5,2])*np.pi, labels = ['0','$\\frac{\pi}{2}$','$\pi$', '$\\frac{\pi}{2}$', '$2\pi$'],fontsize = 14)
    plt.yticks([])
    plt.xlabel('Orientation Angle', fontsize = 14)
#     plt.ylabel('Sorted Neurons', fontsize = 14)
    
    plt.subplot(1,3,3)
    plt.title('Lowest Weights',fontsize = 16)
    Z2_sum3 = np.zeros((Z2.shape[1]))
    for k , k2 in enumerate(np.arange(n_start,n_stop,n_step)):
        if p2.components_[compN,pidx[k]] >=0:
            col = poscol
        else:
            col = negcol
        plt.plot(t,Z2[pidx[-k2-1],idx] + k*ydel, color = col)
        Z2_sum3 = Z2_sum3 + Z2[pidx[-k2-1],idx]
        Z2_sum3 = filtfilt(b,a,Z2_sum3)
    plt.plot(t,(Z2_sum3)*40/((n_stop - n_start)/n_step) + (k+1.5)*ydel, color = [0.5,0,0])
#     plt.ylim([n_start*ydel,(n_stop - n_start)*ydel/n_step])
    plt.ylim([n_start*ydel,(n_stop - n_start + yupperlim)*ydel/n_step])
    plt.xlim([0,2*np.pi])
    plt.xticks(ticks = np.array([0,0.5,1,1.5,2])*np.pi, labels = ['0','$\\frac{\pi}{2}$','$\pi$', '$\\frac{\pi}{2}$', '$2\pi$'],fontsize = 14)
    plt.yticks([])
    plt.xlabel('Orientation Angle', fontsize = 14)
    plt.ylabel('Sorted Neurons', fontsize = 14)
    return None

def plot_neuron_locs(Z2 , dat2 ,p2, compN, n_start = 0, n_stop = 100, n_step = 1):
    """Plot locations of neurons highlighting highest/middle/lowest weighted neurons in red
    
    Args:
    Z2 (numpy.ndarray): Z-scored neural response
    dat2 (dict): loaded stringer dataset
    p2 (PCA model): PCA model based on Z2
    compN (int): Principal component number to sort neurons by their weights
  
    Returns:
    None
    """
    from matplotlib import pyplot as plt
    from scipy.signal import butter, filtfilt
    from scipy.stats import zscore
    b,a = butter(4, [0.1], 'lowpass')
    fig = plt.figure(figsize = (14,4))
    pidx = np.argsort((-p2.components_[compN,:]))
    
    tp = np.zeros((dat2['sresp'].shape[0],2))
    for k in range(dat2['sresp'].shape[0]):
        tp[k,0] = dat2['stat'][k]['xpix'].mean()
        tp[k,1] = dat2['stat'][k]['ypix'].mean()
    
    xrange = np.arange(n_start,n_stop,n_step)

    plt.subplot(1,3,1)
    ax = plt.gca()
    ind1 = pidx[xrange]
    ax.scatter(tp[:,0], tp[:,1], 0.05, color = 'blue')
    ax.scatter(tp[ind1,0], tp[ind1,1], 10, 'red')
    ax.set_facecolor('k')
    ax.get_xaxis().set_visible(False)
    ax.get_yaxis().set_visible(False)
    ax.set_xlim([0, max(tp[:,0])])
    ax.set_ylim([0, max(tp[:,1])])
    
    plt.subplot(1,3,2)
    ax2 = plt.gca()
    ind2 = pidx[xrange + np.round(len(pidx)/2 - (n_stop-n_start)/2).astype(int)]
    ax2.scatter(tp[:,0], tp[:,1], 0.05, color = 'blue')
    ax2.scatter(tp[ind2,0], tp[ind2,1], 10, 'red')
    ax2.set_facecolor('k')
    ax2.get_xaxis().set_visible(False)
    ax2.get_yaxis().set_visible(False)
    ax2.set_xlim([0, max(tp[:,0])])
    ax2.set_ylim([0, max(tp[:,1])])
    
    plt.subplot(1,3,3)
    ax3 = plt.gca()
    ind3 = pidx[-xrange-1]
    ax3.scatter(tp[:,0], tp[:,1], 0.05, color = 'blue')
    ax3.scatter(tp[ind3,0], tp[ind3,1], 10, 'red')
    ax3.set_facecolor('k')
    ax3.get_xaxis().set_visible(False)
    ax3.get_yaxis().set_visible(False)
    ax3.set_xlim([0, max(tp[:,0])])
    ax3.set_ylim([0, max(tp[:,1])])
    return None